In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import polars as pl
import gc

# Pandas의 출력 옵션 설정
pd.set_option('display.max_columns', None)  # 모든 열을 출력
pd.set_option('display.width', None)  # 출력 너비 제한 해제

pl.Config.set_tbl_cols(None)  # 모든 열 출력
pl.Config.set_tbl_rows(None)  # 모든 행 출력

polars.config.Config

# Data Preparing

In [2]:
# Parquet 파일을 읽기
# polar를 활용하여 3번째 까지만 읽어옴
'''
for i in range(0,11,1):
    df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')
    globals()[f'df_{i + 1}'] = df

df_total = pl.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])
df_total.shape
'''

"\nfor i in range(0,11,1):\n    df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')\n    globals()[f'df_{i + 1}'] = df\n\ndf_total = pl.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10])\ndf_total.shape\n"

In [2]:
# build datareading model
def read_data(num_f, num_s = 0):
    dfs = []
    for i in range(num_s,num_f):
        df = pl.read_parquet(f'datasets/train.parquet/partition_id={i}')
        dfs.append(df)

    df_total = pl.concat(dfs)
    return df_total

In [3]:
#df_early_model = read_data(2)
#df_mid_model = read_data(6, num_s = 4)
df_late_model = read_data(10, num_s = 9)

In [5]:
# feature grouping by Tag_n CSV, by distance = 2
# But just before, we gotta do scale the data!


g1 = [44,55,7,60,52,41,49]
g2 = [43, 54, 46, 57, 42, 53]
g3 = [6,59,37,38,5,8]
g4 = [47,58,39,50]
g5 = [40,48,51,45,56]
g6 = [0,1,32,33]
g7 = [2,34,3,4,35,36]
g8 = [20,31,30,29,28,27,26,25,24,23,21,22]
g9 = [70,12,67]
g10 = [71,13,68]
g11 = [72,14,69]
g12 = [73,74,75,76,77,78]
g13 = [17,64,15,62,16,63]
g14 = [61,11,9,10]
g15 = [18,65,19,66]


# Functions - handling data

In [9]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.mixture import GaussianMixture

def date_and_resp(df_tmp_total):
    # responder들 가져요기
    df_tmp_resp = df_tmp_total[:,83:92]

    # 첫 번째 행의 date_id 가져오기
    date_id_value = df_tmp_total["date_id"]

    # date_id 값을 새로운 열로 추가
    df_tmp_resp = df_tmp_resp.with_columns(
        pl.lit(date_id_value).alias("date_id")  # date_id 열 추가
    )

    return df_tmp_resp

def date_and_feature(df_tmp_total):
    # responder들 가져요기
    df_tmp_feat = df_tmp_total[:,4:83]

    # 첫 번째 행의 date_id 가져오기
    date_id_value = df_tmp_total["date_id"]  

    # date_id 값을 새로운 열로 추가
    df_tmp_feat = df_tmp_feat.with_columns(
        pl.lit(date_id_value).alias("date_id")  # date_id 열 추가
    )

    return df_tmp_feat

def sample_df_small(df):
    df_sampled = df.sample(n=2000000, with_replacement=False)
    return df_sampled

In [10]:
test_data = date_and_feature(df_late_model).drop('date_id')

In [8]:
test_data.to_pandas()

,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78
0,1.153571,1.563784,0.697396,0.756759,2.580965,0.171311,1.126353,0.536153,0.057150,11,7,76,-0.656288,2.110188,0.145784,NaN,-0.203291,NaN,-1.238222,-2.294707,-0.063560,-0.148218,1.721362,0.645580,1.477857,0.528492,1.153077,0.466157,0.145568,-0.546845,-0.694435,-0.163897,NaN,NaN,0.502917,0.910145,-0.507707,0.218792,0.412922,NaN,0.081268,NaN,NaN,-2.023247,NaN,-1.967165,0.262769,-0.426009,-3.682122,-1.549827,NaN,0.680807,NaN,NaN,-2.786826,NaN,-1.227900,0.044606,NaN,-2.540213,-2.190280,0.385893,-0.460265,-0.415684,-0.457720,-1.333965,-2.234130,-0.352034,3.125156,0.493488,-0.959100,1.284456,-0.275493,NaN,NaN,4.188457,3.666236,0.848177,0.999516
1,0.553354,1.730064,0.990195,0.611490,2.023031,0.319015,1.183371,0.562853,0.057789,11,7,76,-1.063518,1.037634,-0.255358,NaN,-0.318528,NaN,-1.466130,-2.160217,0.009386,0.042186,0.319811,0.143070,1.866907,1.238242,-1.986826,-0.476918,0.408439,-0.689795,-0.619278,0.081413,NaN,NaN,1.130648,0.726115,2.071485,0.179241,0.045131,NaN,0.002134,NaN,NaN,-0.828163,NaN,-1.304763,0.870251,-0.095340,-0.888243,-0.159577,NaN,-0.002680,NaN,NaN,-1.736226,NaN,-2.354893,1.309985,NaN,-2.429267,-1.266970,0.385893,-0.248770,-0.286104,-0.455154,-1.797363,-2.535985,-0.734866,1.533782,0.033801,-0.960126,0.306505,-0.522036,NaN,NaN,1.138142,1.579439,0.179564,0.160609
2,1.532503,2.095852,0.919688,0.583715,2.330047,0.337096,1.262236,0.496050,0.073556,81,2,59,-1.001967,1.105770,-0.304426,NaN,-0.531873,NaN,-1.301579,-1.615271,0.454406,-0.188808,0.015120,-0.159487,1.379064,0.604568,0.736194,0.522007,-0.183058,-0.632819,-0.839542,-0.209550,NaN,NaN,0.211059,0.788082,-0.575270,0.157013,0.178823,NaN,0.486033,NaN,NaN,-1.121402,NaN,-1.019831,0.741859,-1.735237,-0.707955,-0.510588,NaN,0.793936,NaN,NaN,-1.191118,NaN,-2.190607,1.381697,NaN,-1.829545,-0.867858,0.385893,-0.295958,-0.386221,-0.345102,-1.598371,-2.111468,-0.780465,0.848857,-0.152994,-1.219395,0.359229,-0.636138,NaN,NaN,0.445388,0.300118,-0.043114,-0.065761
3,0.647099,1.687460,0.569406,1.061679,2.444131,0.150487,0.896543,0.705652,0.072545,4,3,11,-0.966606,0.415421,-0.528979,NaN,0.183964,NaN,-1.917427,-1.929856,-0.501750,-0.102371,-0.297677,-1.049185,0.933516,-0.606540,-0.039158,-0.271960,-0.920819,-0.926502,-0.783003,-0.135720,NaN,NaN,1.007292,1.227438,1.663553,-0.243313,-0.049812,NaN,1.388513,NaN,NaN,0.621888,NaN,-0.693391,1.988323,3.209271,5.340614,2.262006,NaN,2.073140,NaN,NaN,0.205406,NaN,-0.760486,2.063410,NaN,4.528522,1.582996,0.385893,-0.396047,-0.146214,-0.228367,-2.062314,-1.421811,-0.851179,0.404792,-0.509694,-1.398495,0.231285,-0.595247,NaN,NaN,6.995454,7.133929,0.352965,0.526284
4,1.096778,1.551411,0.632113,0.368218,2.181873,0.214604,1.194230,0.261056,0.045426,15,1,9,-0.895004,0.047908,-0.683484,NaN,-0.638341,NaN,-1.558110,-1.203378,-0.305831,0.251725,0.222749,-0.902276,4.226925,1.233024,-2.274672,-1.065931,-0.229211,-0.597557,-0.990242,0.173607,NaN,NaN,1.103371,0.061046,-0.951203,-0.034542,-0.000886,NaN,0.743995,NaN,NaN,-0.387709,NaN,-1.331183,1.157520,-0.124650,-0.953787,-0.265533,NaN,0.549144,NaN,NaN,0.383838,NaN,-1.490257,2.019752,NaN,1.872912,0.571886,0.385893,-0.445097,-0.347954,-0.470278,-1.780053,-2.29

# Functions - Clustering
- with no scaling
- impute NaNs with 0

In [12]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def kmeans(data, nclusters = 5, transpose = 'y'):

    col_names = data.columns

    imputer = SimpleImputer(strategy='constant', fill_value=0)
    data_imputed = imputer.fit_transform(data.select(pl.col("*")).to_numpy())
    
    if transpose == 'y':
        data_imputed = data_imputed.transpose() # 또는 그냥 .T로 transpose 가능?

    # K-Means 수행
    kmeans = KMeans(n_clusters=nclusters, random_state=42)
    clusters = kmeans.fit_predict(data_imputed)

    data_res = pl.DataFrame(data_imputed)
    data_res = data_res.with_columns(pl.Series(name = 'Cluster', values = clusters))

    return data_res


In [10]:
kmeans_clustered_dat = kmeans(test_data)

In [11]:
kmeans_clustered_dat['Cluster'].value_counts()

Cluster,count
i32,u32
4,61
2,1
1,1
0,8
3,8


- and now with Standard Scaling

In [13]:
def kmeans_scaled(data, nclusters = 5, transpose = 'y'):

    col_names = data.columns

    transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value=0)), #결측치는 모두 0으로 처리 사유 -> 모두 NaN인 경우 열이 삭제됨.
        ('scaler', StandardScaler())
    ])
    
    data_preprocessed = transformer.fit_transform(data.select(pl.col("*")).to_numpy())
    data_preprocessed = pl.DataFrame(data_preprocessed, schema = col_names)
    
    if transpose == 'y':
        data_preprocessed = data_preprocessed.transpose() # 또는 그냥 .T로 transpose 가능?

    # K-Means 수행
    kmeans = KMeans(n_clusters=nclusters, random_state=42)
    clusters = kmeans.fit_predict(data_preprocessed)

    data_res = pl.DataFrame(data_preprocessed)
    data_res = data_res.with_columns(pl.Series(name = 'Cluster', values = clusters))
    data_res = data_res.with_columns(pl.Series(name = 'Colnames', values = col_names))

    return data_res

In [14]:
kmeans_clustered_dat_scaled = kmeans_scaled(test_data, nclusters = 15)

In [15]:
import pandas as pd

# Polars DataFrame을 Pandas DataFrame으로 변환
df_pandas = kmeans_clustered_dat_scaled['Cluster'].value_counts().sort(by='count').to_pandas()

# Pandas의 출력 설정을 변경하여 전체 데이터 보기
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력

# 전체 출력
df_pandas

,Cluster,count
0,8,2
1,4,2
2,14,3
3,11,3
4,12,4
5,6,4
6,0,4
7,7,5
8,9,5
9,5,6


In [19]:
result = kmeans_clustered_dat_scaled.filter(pl.col('Cluster') == 1)[['Colnames', 'Cluster']]
print(result)

: 

# Clustering with 